In [31]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint




from time import time
from IPython.core.display import clear_output
from warnings import warn





In [32]:
url = "https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start=1&ref_=adv_nxt"
response = requests.get(url)
response.status_code

200

In [33]:
#Connecting with Beautiful Soap for IMDb results
page = response.text
soup = BeautifulSoup(page, "lxml")
soup.prettify()

'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <script type="text/javascript">\n   var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};\n  </script>\n  <script>\n   if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n  </script>\n  <script>\n   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);\n  </script>\n  <title>\n   Released between 2020-01-01 and 2020-12-31\n(Sorted by Number of Votes Descending) - IMDb\n  </title>\n  <script>\n   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);\n  </script>\n  <script>\n   if (typeof uet == \'function\') {\n      uet("be", "LoadTitle", {wb: 1});\n    }\n  </script>\n  <script>\n   if (typeof uex == \'function\') {\n      uex("ld", "LoadTitle", {wb: 1});\n 

In [34]:
# Creating the lists we want to write into
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []
revenue = []



In [35]:
pages = np.arange(1, 3051, 50)
pages

array([   1,   51,  101,  151,  201,  251,  301,  351,  401,  451,  501,
        551,  601,  651,  701,  751,  801,  851,  901,  951, 1001, 1051,
       1101, 1151, 1201, 1251, 1301, 1351, 1401, 1451, 1501, 1551, 1601,
       1651, 1701, 1751, 1801, 1851, 1901, 1951, 2001, 2051, 2101, 2151,
       2201, 2251, 2301, 2351, 2401, 2451, 2501, 2551, 2601, 2651, 2701,
       2751, 2801, 2851, 2901, 2951, 3001])

In [29]:
# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}

In [36]:
# 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start=' + str(page) + '&ref_=adv_nxt', headers = headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)
        
        
        # Scrape the gross revenue
        rev = container.find('span', attrs = {'name':'nv'}).find_next("span").find_next("span").find_next("span").text
        revenue.append(rev)

AttributeError: 'NoneType' object has no attribute 'text'

In [9]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

movies.head()

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,(2021),155 min,8.3,75,"87,012",-
1,Halloween,(1978),91 min,7.7,87,"248,959",$47.00M
2,Casino Royale,(2006),144 min,8.0,80,"614,132",$167.45M
3,Shang-Chi and the Legend of the Ten Rings,(2021),132 min,7.9,71,"109,897",-
4,Skyfall,(2012),143 min,7.8,81,"659,200",$304.36M


In [20]:
movies.isnull().sum()

movie              0
year               0
time_minute        0
imdb_rating        0
metascore        163
vote               0
gross_earning    179
dtype: int64

In [10]:
movies.dtypes

movie             object
year              object
time_minute       object
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

In [11]:
movies.shape

(1000, 7)

In [12]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155 min,8.3,75,"87,012",-
1,Halloween,1978,91 min,7.7,87,"248,959",$47.00M
2,Casino Royale,2006,144 min,8.0,80,"614,132",$167.45M


In [13]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.3,75,"87,012",-
1,Halloween,1978,91,7.7,87,"248,959",$47.00M
2,Casino Royale,2006,144,8.0,80,"614,132",$167.45M


In [14]:
# Cleaning 'metascore' column
movies['metascore'] = movies['metascore'].str.extract('(\d+)')
# convert it to float and if there are dashes turn it into NaN
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

In [15]:
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.3,75.0,"87,012",-
1,Halloween,1978,91,7.7,87.0,"248,959",$47.00M
2,Casino Royale,2006,144,8.0,80.0,"614,132",$167.45M


In [16]:
# Cleaning 'vote' column
movies['vote'] = movies['vote'].str.replace(',', '').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.3,75.0,87012,-
1,Halloween,1978,91,7.7,87.0,248959,$47.00M
2,Casino Royale,2006,144,8.0,80.0,614132,$167.45M


In [17]:
# Cleaning 'gross_earning' column
# left strip $ and right strip M 
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').rstrip('M'))
# convert it to float and if there are dashes turn it into NaN
movies['gross_earning'] = pd.to_numeric(movies['gross_earning'], errors='coerce')
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.3,75.0,87012,NaN
1,Halloween,1978,91,7.7,87.0,248959,47.00
2,Casino Royale,2006,144,8.0,80.0,614132,167.45


In [18]:
movies

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.3,75.0,87012,NaN
1,Halloween,1978,91,7.7,87.0,248959,47.00
2,Casino Royale,2006,144,8.0,80.0,614132,167.45
3,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,109897,NaN
4,Skyfall,2012,143,7.8,81.0,659200,304.36
...,...,...,...,...,...,...,...
995,"Carry On, Munna Bhai",2006,144,8.1,NaN,45372,2.22
996,Anand,1971,122,8.2,NaN,32875,NaN
997,Vizontele,2001,110,8.0,NaN,35314,NaN
998,Kai Po Che,2013,120,7.8,40.0,35428,1.12


In [19]:
movies.to_csv('movies.csv')